# Beyond The Crosshair

In [8]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt

## Datasets

### Counter-Strike Playerbase Data from SteamDB

In [16]:
cs_data = pd.read_csv("../assets/SteamDB Counter-Strike.csv")
cs2_data = pd.read_csv("../assets/SteamDB Counter-Strike 2.csv")
css_data = pd.read_csv("../assets/SteamDB Counter-Strike Source.csv")
cscz_data = pd.read_csv("../assets/SteamDB Counter-Strike Condition Zero.csv")

### Rival Game Data from IGDB

In [17]:
game_data = pd.read_csv("../assets/igdb_shooters.csv")

### Twitch Viewership Data from TwitchTracker

In [18]:
twitch_data = pd.read_csv("../assets/twitchtracker counterstrike2.csv")

### Youtube Video Data from Youtube Data API

In [19]:
youtube_data = pd.read_csv("../assets/yt_counter_strike.csv")

In [38]:
cs_data_with_dash = cs_data[cs_data["gain"] == "-"]
cs_data_with_dash

,month,peak,gain,%gain,average,average % gain,Counter-Strike
241,Jan-04,"23,241",-,-,-,-,yes


In [39]:
cs_data[['peak', 'gain', 'average']] = cs_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
cs2_data[['peak', 'gain', 'average']] = cs2_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
css_data[['peak', 'gain', 'average']] = css_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)
cscz_data[['peak', 'gain', 'average']] = cscz_data[['peak', 'gain', 'average']].replace({",": "", "-": "0"}, regex=True).astype(float)

In [47]:
fields_game_versions = ['Counter-Strike', 'Counter-Strike 2', 'Counter-Strike Source', 'Counter-Strike Condition Zero']
fields_to_keep = ['month', 'peak', 'gain', 'average'] + fields_game_versions
cs_data['Counter-Strike'] = 'yes'
cs2_data['Counter-Strike 2'] = 'yes'
css_data['Counter-Strike Source'] = 'yes'
cscz_data['Counter-Strike Condition Zero'] = 'yes'
cs_merged = pd.concat([cs_data.iloc[1:], cs2_data.iloc[1:], css_data.iloc[1:], cscz_data.iloc[1:]], ignore_index=True)
cs_merged = cs_merged[fields_to_keep]
cs_merged.head()

,month,peak,gain,average,Counter-Strike,Counter-Strike 2,Counter-Strike Source,Counter-Strike Condition Zero
0,Aug-25,12680.0,1946.0,7007.0,yes,NaN,NaN,NaN
1,Jul-25,14626.0,1104.0,7412.0,yes,NaN,NaN,NaN
2,Jun-25,15730.0,81.0,8251.0,yes,NaN,NaN,NaN
3,May-25,15811.0,2394.0,9224.0,yes,NaN,NaN,NaN
4,Apr-25,18205.0,430.0,9653.0,yes,NaN,NaN,NaN


In [49]:
cs_merged_grouped = cs_merged.groupby(cs_merged.columns[0]).sum()
cs_merged_grouped[fields_game_versions] = cs_merged_grouped[fields_game_versions].replace(0, "no")
cs_merged_grouped

,peak,gain,average,Counter-Strike,Counter-Strike 2,Counter-Strike Source,Counter-Strike Condition Zero
month,,,,,,,
Apr-04,84182.0,35217.0,0.0,yes,no,no,yes
Apr-05,144177.0,24782.0,0.0,yes,no,yes,yes
Apr-07,131302.0,114236.0,0.0,yes,no,yes,yes
Apr-09,115364.0,64362.0,0.0,yes,no,yes,yes
Apr-10,167923.0,8719.0,0.0,yes,no,yes,yes
...,...,...,...,...,...,...,...
Sep-20,1003755.0,49306.0,0.0,yes,yes,yes,yes
Sep-21,970788.0,146610.0,0.0,yes,yes,yes,yes
Sep-22,1124191.0,63558.0,0.0,yes,yes,yes,yes


In [55]:
cs_merged_grouped.to_csv("../assets/cs_merged_clean.csv",encoding="utf-8")